# Advanced Querying Mongo

Importing libraries and setting up connection

In [42]:
!pip install pymongo
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [68]:
import pandas as pd

In [51]:
client.list_database_names()

['admin', 'companies', 'config', 'local']

In [52]:
db= client.companies

In [56]:
colec= db.companies

In [54]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [57]:
colec

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [62]:
list(colec.find(  {'name': 'Babelgum'}, 
               
                  {'_id': 0, 'name': 1}
               
               ))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [141]:
df=pd.DataFrame(list(colec.find(  {'number_of_employees': {'$gt':5000}} ).sort('number_of_employees', -1).limit(20) 
    ))
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd67529941a,Siemens,siemens,http://www.crunchbase.com/company/siemens,http://www.siemens.com,,,Siemens,hardware,405000,...,[],None,"[{'price_amount': 418000000, 'price_currency_c...","[{'description': '', 'address1': 'Wittelsbache...","[{'id': 35852, 'description': 'Siemens Receive...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<iframe width=""420"" height=""3...","[{'available_sizes': [[[150, 92], 'assets/imag...",[],[]
1,52cdef7c4bab8bd67529856a,IBM,ibm,http://www.crunchbase.com/company/ibm,http://www.ibm.com,,,IBM,software,388000,...,"[{'funding_round': {'round_code': 'c', 'source...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Headquarters', 'ad...","[{'id': 10471, 'description': 'IBM Completes A...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 93], 'assets/imag...","[{'external_url': 'http://mashpedia.com/IBM', ...",[]
2,52cdef7d4bab8bd675299d33,Toyota,toyota,http://www.crunchbase.com/company/toyota,http://www.toyota-global.com,,,Toyota,enterprise,320000,...,"[{'funding_round': {'round_code': 'a', 'source...",None,[],"[{'description': 'HQ', 'address1': 'Toyota Mot...",[],None,"[{'embed_code': '<iframe width=""430"" height=""3...","[{'available_sizes': [[[150, 84], 'assets/imag...",[],[]
3,52cdef7c4bab8bd675297e89,PayPal,paypal,http://www.crunchbase.com/company/paypal,http://www.paypal.com,,,paypal,finance,300000,...,"[{'funding_round': {'round_code': 'c', 'source...","{'price_amount': 1500000000, 'price_currency_c...","[{'price_amount': None, 'price_currency_code':...","[{'description': '', 'address1': '2145 E Hamil...","[{'id': 11916, 'description': 'Reserve Bank Of...",None,[],"[{'available_sizes': [[[150, 120], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[]
4,52cdef7e4bab8bd67529b0fe,Nippon Telegraph and Telephone Corporation,nippon-telegraph-and-telephone-corporation,http://www.crunchbase.com/company/nippon-teleg...,http://www.ntt.co.jp/index_e.html,,,,None,227000,...,"[{'funding_round': {'round_code': 'e', 'source...",None,[],"[{'description': 'NTT', 'address1': '3-1, Otem...",[],None,[],"[{'available_sizes': [[[150, 42], 'assets/imag...",[],[]
5,52cdef7d4bab8bd675298aa4,Samsung Electronics,samsung-electronics,http://www.crunchbase.com/company/samsung-elec...,http://www.samsung.com/us,http://www.samsungvillage.com/,,samsungtweets,hardware,221726,...,"[{'funding_round': {'round_code': 'b', 'source...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'HQ - South Korea', 'address1...","[{'id': 11908, 'description': 'Samsung Ships 3...",None,"[{'embed_code': '<iframe src=""http://blip.tv/p...","[{'available_sizes': [[[94, 150], 'assets/imag...",[{'external_url': 'http://mashpedia.com/Samsun...,[]
6,52cdef7d4bab8bd675298b99,Accenture,accenture,http://www.crunchbase.com/company/accenture,http://www.accenture.com,http://www.accenture.com/Global/Accenture_Blogs/,http://feeds.accenture.com/Accenture_High_Perf...,accenture,consulting,205000,...,[{'funding_round': {'round_code': 'unattribute...,None,"[{'price_amount': None, 'price_currency_code':...","[{'description': '', 'address1': '1615 Murray ...","[{'id': 1824, 'description': 'Nokia and Accent...","{'valuation_amount': None, 'valuation_currency...",[],"[{'available_sizes': [[[150, 136], 'assets/ima...",[{'external_url': 'http://www.accenture.com/Gl...,[]
7,52cdef7e4bab8bd67529a657,Tata Consultancy Services,tata-consultancy-services,http://www.crunchbase.com/company/tata-consult...,http://www.tcs.com,,http://www.tcs.com/rss_feeds/Pages/feed.aspx?f=p,TCS_News,consulting,200300,...,[],None,[],"[{'description': 'World Headquarte

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [111]:
df=pd.DataFrame(list(colec.find({'$and': [{'founded_year': {'$gte': 2000}}, {'founded_year':{'$lte': 2005}}] }, 
  {'_id': 0, 'name': 1, 'founded_year': 1}
)))
df.head()

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Digg,2004
3,Facebook,2004
4,Omnidrive,2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [115]:
df=pd.DataFrame(list
               (colec.find(
               {'$and': [{"ipo.valuation_amount": {'$gt': 10000000}}, {'founded_year':{'$lt': 2010}}] }, 
               {'_id': 0, 'name': 1, 'ipo': 1}
)))
df.head()

,name,ipo
0,Facebook,"{'valuation_amount': 104000000000, 'valuation_..."
1,Twitter,"{'valuation_amount': 18100000000, 'valuation_c..."
2,Yelp,"{'valuation_amount': 1300000000, 'valuation_cu..."
3,LinkedIn,"{'valuation_amount': 9310000000, 'valuation_cu..."
4,Amazon,"{'valuation_amount': 100000000000, 'valuation_..."


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [140]:
df=pd.DataFrame(list
               (colec.find(
               {'$and': [{'number_of_employees': {'$lt':1000}}, {'founded_year': {'$lt': 2005}}] }
               ).sort('number_of_employees', -1).limit(10)
               ))
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd675298933,Infinera Corporation,infinera,http://www.crunchbase.com/company/infinera,http://www.infinera.com,,,InfineraCorp,network_hosting,974,...,[],None,[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 20267, 'description': 'Infinera PICs S...","{'valuation_amount': None, 'valuation_currency...",[],"[{'available_sizes': [[[150, 112], 'assets/ima...",[],[]
1,52cdef7e4bab8bd67529ac95,NorthPoint Communications Group,northpoint-communications-group,http://www.crunchbase.com/company/northpoint-c...,,,,,network_hosting,948,...,[],None,[],"[{'description': 'HQ', 'address1': '303 Second...",[],None,[],[],[],[]
2,52cdef7f4bab8bd67529be17,888 Holdings,888-holdings,http://www.crunchbase.com/company/888-holdings,http://www.888holdingsplc.com,,,,games_video,931,...,[],None,"[{'price_amount': 18000000, 'price_currency_co...",[],[],None,[],"[{'available_sizes': [[[143, 150], 'assets/ima...",[],[]
3,52cdef7c4bab8bd6752986a2,Forrester Research,forrester-research,http://www.crunchbase.com/company/forrester-re...,http://forrester.com,http://blogs.forrester.com/colony/,http://blogs.forrester.com/colony/index.rdf,,consulting,903,...,[],None,"[{'price_amount': 4750000, 'price_currency_cod...","[{'description': None, 'address1': '400 Techno...","[{'id': 36801, 'description': 'Forrester: $2.1...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 93], 'assets/imag...",[{'external_url': 'http://blogs.forrester.com/...,[]
4,52cdef7e4bab8bd67529af6d,SonicWALL,sonicwall,http://www.crunchbase.com/company/sonicwall,http://www.sonicwall.com,,,sonicwall,security,900,...,[],"{'price_amount': 717000000, 'price_currency_co...","[{'price_amount': 25000000, 'price_currency_co...","[{'description': 'HQ', 'address1': '2001 Logic...","[{'id': 15615, 'description': 'SonicWALL Revol...",None,"[{'embed_code': '<iframe width=""480"" height=""3...",[],[],[]
5,52cdef7e4bab8bd67529b21b,Webmetrics,webmetrics,http://www.crunchbase.com/company/webmetrics,http://www.webmetrics.com,http://blog.webmetrics.com,http://feeds.feedburner.com/webmetricsperforma...,Webmetrics,software,900,...,[],None,[],"[{'description': '', 'address1': '9444 Waples'...",[],None,[],"[{'available_sizes': [[[150, 125], 'assets/ima...",[],[]
6,52cdef7e4bab8bd67529b3bd,Cornerstone OnDemand,cornerstone-ondemand,http://www.crunchbase.com/company/cornerstone-...,http://www.cornerstoneondemand.com,http://www.cornerstoneondemand.com/blog,http://www.cornerstoneondemand.com/blog,cornerstoneinc,software,881,...,[],None,[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 7141, 'description': 'Announced hire o...","{'valuation_amount': 900000000, 'valuation_cur...",[],[],[],[]
7,52cdef7c4bab8bd675297ff0,Mozilla,mozilla,http://www.crunchbase.com/company/mozilla,http://mozilla.org,http://blog.mozilla.org,http://blog.mozilla.org/feed/,firefox,web,800,...,[{'funding_round': {'round_code': 'unattribute...,None,[],"[{'description': '', 'address1': '650 Castro S...","[{'id': 153, 'description': 'UnqVjH srogpklot...",None,"[{'embed_code': '<div style=""width:320px;""><di...",[],[{'external_url': 'http://mashpedia.com/Mozill...,[]
8,52cdef7c4bab8bd67529857e,Buongiorno,buongiorno,http://www.crunchbase.com/company/buongiorno,http://www.buongiorno.com,,,AnnPRBuongiorno,mobile,800,...,[],None,"[{'price_amount': None, 'price_currency_code':...","[{'description': '', 'address1': 'Via Cosimo d...","[{'id': 23902, 'description': 'NTT DOCOMO, INC...","{'valuation_amount': None, 'valuation_currency...",[],[],[],[]
9,52cdef7c4bab8bd675297de0,Yelp,yelp,http://www.crunchbase.com/company/yelp,http://yelp.com,http://officialblog.yelp.com/,http://naturallylindy.com,Yelp,search,800,..

### 6. All the companies that don't include the `partners` field.

In [124]:
df=pd.DataFrame(list
               (colec.find({'partners': {'$size': 0}})
               ))
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,"[{'description': 'Headquarters', 'address1': '...",[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'description': 'Headquarters', 'address1': '...","[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],NaN,NaN,NaN,NaN
3,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,"[{'description': None, 'address1': '135 Missis...","[{'id': 9588, 'description': 'Another Digg Exe...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[117, 150], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[],NaN,NaN,None,None
4,52cdef7c4bab8bd675297d8e,Facebook,facebook,http://www.crunchbase.com/company/facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299.0,...,"[{'description': 'Headquarters', 'address1': '...","[{'id': 108, 'description': 'Facebook adds com...",[],"[{'available_sizes': [[[150, 68], 'assets/imag...",[{'external_url': 'http://latimesblogs.latimes...,[],NaN,NaN,,"{'valuation_amount': 104000000000, 'valuation_..."


### 7. All the companies that have a null type of value on the `category_code` field.

In [129]:
df=pd.DataFrame(list
               (colec.find({ 'category_code': { '$type': "null" } })
               ))
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd6752980f6,Collective,collective,http://www.crunchbase.com/company/collective,None,None,None,None,None,NaN,...,[],None,[],[],[],None,[],[],[],[]
1,52cdef7c4bab8bd675298225,Snimmer,snimmer,http://www.crunchbase.com/company/snimmer,None,None,None,None,None,NaN,...,[],None,[],[],[],None,[],[],[],[]
2,52cdef7c4bab8bd675298226,KoolIM,koolim,http://www.crunchbase.com/company/koolim,None,None,None,None,None,NaN,...,[],None,[],[],[],None,[],[],[],[]
3,52cdef7c4bab8bd675298261,Level9 Media,level9-media,http://www.crunchbase.com/company/level9-media,None,None,None,None,None,NaN,...,[],"{'price_amount': None, 'price_currency_code': ...",[],[],[],None,[],[],[],[]
4,52cdef7c4bab8bd675298262,VidKing,vidking,http://www.crunchbase.com/company/vidking,http://www.VidKing.com,,,None,None,NaN,...,[],"{'price_amount': 150000, 'price_currency_code'...",[],[],[],None,[],[],[],[]


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [131]:
df=pd.DataFrame(list
               (colec.find(
              { '$and': [{'number_of_employees': {'$gte': 100}}, {'number_of_employees': {'$lt': 1000}}]}, 
              {'_id': 0, 'name': 1, 'number_of_employees': 1})
))
df.head()

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [135]:
df=pd.DataFrame(list
               (colec.find().sort("ipo.valuation_amount", -1)
               ))
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7e4bab8bd67529a8b4,GREE,gree,http://www.crunchbase.com/company/gree,http://www.gree-corp.com,,,gree_corp,games_video,700.0,...,"[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': 104000000, 'price_currency_c...","[{'description': 'Headquarters', 'address1': '...","[{'id': 1021, 'description': 'IPO', 'stoned_ye...","{'valuation_amount': 108960000000, 'valuation_...",[],"[{'available_sizes': [[[133, 150], 'assets/ima...",[],[]
1,52cdef7c4bab8bd675297d8e,Facebook,facebook,http://www.crunchbase.com/company/facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299.0,...,"[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Headquarters', 'address1': '...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...",[],"[{'available_sizes': [[[150, 68], 'assets/imag...",[{'external_url': 'http://latimesblogs.latimes...,[]
2,52cdef7c4bab8bd675297e7a,Amazon,amazon,http://www.crunchbase.com/company/amazon,http://amazon.com,,,amazon,ecommerce,NaN,...,"[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': None, 'address1': '1200 12th ...","[{'id': 6931, 'description': 'How Many Kindle ...","{'valuation_amount': 100000000000, 'valuation_...","[{'embed_code': '<object width=""425"" height=""3...","[{'available_sizes': [[[150, 94], 'assets/imag...",[],[]
3,52cdef7c4bab8bd675297d94,Twitter,twitter,http://www.crunchbase.com/company/twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300.0,...,[],None,"[{'price_amount': 15000000, 'price_currency_co...","[{'description': '', 'address1': '1355 Market ...","[{'id': 1, 'description': 'Survives iPhone 3G ...","{'valuation_amount': 18100000000, 'valuation_c...","[{'embed_code': '<iframe width=""430"" height=""2...","[{'available_sizes': [[[150, 74], 'assets/imag...",[{'external_url': 'http://sec.gov/cgi-bin/brow...,[]
4,52cdef7d4bab8bd675299d5d,Groupon,groupon,http://www.crunchbase.com/company/groupon,http://www.groupon.com,http://www.groupon.com/blog/,feed://feeds2.feedburner.com/grouponblog,groupon,web,10000.0,...,[{'funding_round': {'round_code': 'unattribute...,None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'HQ', 'address1': '600 W. Chi...","[{'id': 4396, 'description': 'Groupon Makes A ...","{'valuation_amount': 12800000000, 'valuation_c...","[{'embed_code': '<script src=""http://www.buzzs...","[{'available_sizes': [[[150, 97], 'assets/imag...",[{'external_url': 'http://web20show.com/2009/1...,[]


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [139]:
df=pd.DataFrame(list
               (colec.find().sort('number_of_employees', -1).limit(10)
               ))
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd67529941a,Siemens,siemens,http://www.crunchbase.com/company/siemens,http://www.siemens.com,,,Siemens,hardware,405000,...,[],None,"[{'price_amount': 418000000, 'price_currency_c...","[{'description': '', 'address1': 'Wittelsbache...","[{'id': 35852, 'description': 'Siemens Receive...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<iframe width=""420"" height=""3...","[{'available_sizes': [[[150, 92], 'assets/imag...",[],[]
1,52cdef7c4bab8bd67529856a,IBM,ibm,http://www.crunchbase.com/company/ibm,http://www.ibm.com,,,IBM,software,388000,...,"[{'funding_round': {'round_code': 'c', 'source...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Headquarters', 'ad...","[{'id': 10471, 'description': 'IBM Completes A...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 93], 'assets/imag...","[{'external_url': 'http://mashpedia.com/IBM', ...",[]
2,52cdef7d4bab8bd675299d33,Toyota,toyota,http://www.crunchbase.com/company/toyota,http://www.toyota-global.com,,,Toyota,enterprise,320000,...,"[{'funding_round': {'round_code': 'a', 'source...",None,[],"[{'description': 'HQ', 'address1': 'Toyota Mot...",[],None,"[{'embed_code': '<iframe width=""430"" height=""3...","[{'available_sizes': [[[150, 84], 'assets/imag...",[],[]
3,52cdef7c4bab8bd675297e89,PayPal,paypal,http://www.crunchbase.com/company/paypal,http://www.paypal.com,,,paypal,finance,300000,...,"[{'funding_round': {'round_code': 'c', 'source...","{'price_amount': 1500000000, 'price_currency_c...","[{'price_amount': None, 'price_currency_code':...","[{'description': '', 'address1': '2145 E Hamil...","[{'id': 11916, 'description': 'Reserve Bank Of...",None,[],"[{'available_sizes': [[[150, 120], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[]
4,52cdef7e4bab8bd67529b0fe,Nippon Telegraph and Telephone Corporation,nippon-telegraph-and-telephone-corporation,http://www.crunchbase.com/company/nippon-teleg...,http://www.ntt.co.jp/index_e.html,,,,None,227000,...,"[{'funding_round': {'round_code': 'e', 'source...",None,[],"[{'description': 'NTT', 'address1': '3-1, Otem...",[],None,[],"[{'available_sizes': [[[150, 42], 'assets/imag...",[],[]
5,52cdef7d4bab8bd675298aa4,Samsung Electronics,samsung-electronics,http://www.crunchbase.com/company/samsung-elec...,http://www.samsung.com/us,http://www.samsungvillage.com/,,samsungtweets,hardware,221726,...,"[{'funding_round': {'round_code': 'b', 'source...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'HQ - South Korea', 'address1...","[{'id': 11908, 'description': 'Samsung Ships 3...",None,"[{'embed_code': '<iframe src=""http://blip.tv/p...","[{'available_sizes': [[[94, 150], 'assets/imag...",[{'external_url': 'http://mashpedia.com/Samsun...,[]
6,52cdef7d4bab8bd675298b99,Accenture,accenture,http://www.crunchbase.com/company/accenture,http://www.accenture.com,http://www.accenture.com/Global/Accenture_Blogs/,http://feeds.accenture.com/Accenture_High_Perf...,accenture,consulting,205000,...,[{'funding_round': {'round_code': 'unattribute...,None,"[{'price_amount': None, 'price_currency_code':...","[{'description': '', 'address1': '1615 Murray ...","[{'id': 1824, 'description': 'Nokia and Accent...","{'valuation_amount': None, 'valuation_currency...",[],"[{'available_sizes': [[[150, 136], 'assets/ima...",[{'external_url': 'http://www.accenture.com/Gl...,[]
7,52cdef7e4bab8bd67529a657,Tata Consultancy Services,tata-consultancy-services,http://www.crunchbase.com/company/tata-consult...,http://www.tcs.com,,http://www.tcs.com/rss_feeds/Pages/feed.aspx?f=p,TCS_News,consulting,200300,...,[],None,[],"[{'description': 'World Headquarte

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [145]:
df=pd.DataFrame(list
               (colec.find({ 'founded_month': {'$gte': 7} }).limit(1000)
               ))
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'description': 'Headquarters', 'address1': '...","[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,"[{'description': None, 'address1': '135 Missis...","[{'id': 9588, 'description': 'Another Digg Exe...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[117, 150], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[],NaN,NaN,None,None
3,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,...,"[{'description': '', 'address1': 'Suite 200', ...",[],[],[],[],[],9.0,15.0,,None
4,52cdef7c4bab8bd675297d9b,eBay,ebay,http://www.crunchbase.com/company/ebay,http://ebay.com,,,eBay,web,15000.0,...,"[{'description': 'Headquarters', 'address1': '...","[{'id': 7107, 'description': 'eBay Q2 Revenue ...","[{'embed_code': '<iframe width=""370"" height=""3...","[{'available_sizes': [[[150, 71], 'assets/imag...",[],[],NaN,NaN,None,"{'valuation_amount': None, 'valuation_currency..."


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [148]:
df=pd.DataFrame(list
               (colec.find({'$and': [{'founded_year': {'$lt': 2000}}, {"acquisition.price_amount": {'$gt': 10000000}}] })
               ))
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,[],"{'price_amount': 625000000, 'price_currency_co...",[],"[{'description': None, 'address1': '959 Skyway...",[],None,[],[],[],[]
1,52cdef7c4bab8bd675297deb,SideStep,sidestep,http://www.crunchbase.com/company/sidestep,http://www.sidestep.com,None,None,None,web,75.0,...,[],"{'price_amount': 180000000, 'price_currency_co...",[],"[{'description': None, 'address1': '2980 Bower...",[],None,[],[],[],[]
2,52cdef7c4bab8bd675297e2c,Recipezaar,recipezaar,http://www.crunchbase.com/company/recipezaar,http://www.recipezaar.com,http://www.recipezaar.com/sitenews/,None,None,web,5.0,...,[],"{'price_amount': 25000000, 'price_currency_cod...",[],"[{'description': None, 'address1': 'P.O. Box 1...",[],None,[],[],[],[]
3,52cdef7c4bab8bd675297e89,PayPal,paypal,http://www.crunchbase.com/company/paypal,http://www.paypal.com,,,paypal,finance,300000.0,...,"[{'funding_round': {'round_code': 'c', 'source...","{'price_amount': 1500000000, 'price_currency_c...","[{'price_amount': None, 'price_currency_code':...","[{'description': '', 'address1': '2145 E Hamil...","[{'id': 11916, 'description': 'Reserve Bank Of...",None,[],"[{'available_sizes': [[[150, 120], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[]
4,52cdef7c4bab8bd675297e97,Snapfish,snapfish,http://www.crunchbase.com/company/snapfish,http://snapfish.com,http://snapfishblog.in/,,snapfishbyhp,network_hosting,NaN,...,[],"{'price_amount': 300000000, 'price_currency_co...","[{'price_amount': None, 'price_currency_code':...","[{'description': None, 'address1': '303 Second...",[],None,[],[],[{'external_url': 'http://www.upickreviews.com...,[]


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code